# 1. 크롤링 데이터 전처리<br>
Scrapy를 통해 List 형식으로 크롤링된 시를 str 형식으로 변경

In [1]:
import re
import pandas as pd
import ast
import hanja

In [ ]:
from_teen = pd.read_csv("../raw_data/geulteen_poems.csv").drop("Unnamed: 0", axis=1)
from_modern_poems = pd.read_csv("../raw_data/modern_poems_raw.csv").drop("Unnamed: 0", axis=1)
from_dica_poems = pd.read_csv("../raw_data/dica_poems_raw.csv").drop("Unnamed: 0", axis=1)

In [ ]:
# "[poem]" 형식의 str을 List인 [poem]으로 치환
def listify(poem):
    return ast.literal_eval(poem)

In [ ]:
from_teen['poem'] = from_teen['poem'].apply(listify)
from_modern_poems['poem'] = from_modern_poems['poem'].apply(listify)
from_dica_poems['poem'] = from_dica_poems['poem'].apply(listify)

In [ ]:
# 리스트 안의 시 내용을 개행문자로 묶어줌. 
def strip_and_join_newline(poem_list):
    return "\n".join(map(str.strip, poem_list))

# 2. 데이터의 노이즈가 될 수 있는 부분 전처리
KLUE 데이터셋의 전처리 방식을 활용

In [ ]:
# Klue 데이터셋 전처리 응용
def preprocess(poem):
    new = []
    for text in poem:
        # 문제를 일으킬 수 있는 문자 제거
        bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
        for bad_char in bad_chars:
            text = text.replace(bad_char, bad_chars[bad_char])
            
        error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0":" "}
        for error_char in error_chars:
            text = text.replace(error_char, error_chars[error_char])

        # URL 제거
        text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "[웹주소]", text).strip()
        text = re.sub(r"pic\.(\w+\.)+\S*", "[웹주소]", text).strip()

        # 뉴스 저작권 관련 텍스트 제거
        re_patterns = [
            r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
            r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))"
        ]
        
        for re_pattern in re_patterns:
            text = re.sub(re_pattern, "", text).strip()
        
        # 뉴스 내 포함된 이미지에 대한 레이블 제거
        text = re.sub(r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ", "", text).strip()
        
        # 문제를 일으킬 수 있는 구두점 치환
        punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
        for p in punct_mapping:
            text = text.replace(p, punct_mapping[p])
        
        # 연속된 공백 치환
        text = re.sub(r"\s+", " ", text).strip()
        
        # 개행을 먼저 없애고 그 후 합쳐줌.
        re.sub('\n|\t|\r', "", text).strip()
        re.sub('\xa0', " ", text).strip()

        if text:
            new.append(text)
    return "\n".join(new)

In [ ]:
from_teen['poem'] = from_teen['poem'].apply(preprocess)
from_modern_poems['poem'] = from_modern_poems['poem'].apply(preprocess)
from_dica_poems['poem'] = from_dica_poems['poem'].apply(preprocess)

# 3. 한자 전처리
한자 전처리가 없을 시, 모델이 적절하지 않은 한자어를 생성하는 경우가 있음을 확인, 데이터의 한자를 제거 및 번역

In [ ]:
# 한글과 병기된 한자어는 삭제 후, 남은 한자어는 번역
def hanja_preprocess(txt):
    new = re.sub(r"\([\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff]+\)|\[[\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff]+\]", "", txt)
    new = hanja.translate(new, 'substitution')
    return new

In [ ]:
from_teen['poem'] = from_teen['poem'].apply(hanja_preprocess)
from_modern_poems['poem'] = from_modern_poems['poem'].apply(hanja_preprocess)
from_dica_poems['poem'] = from_dica_poems['poem'].apply(hanja_preprocess)

# 4. CSV 형식으로 Export
같은 형식으로 응용 가능한 teen과 modern_poems는 하나로 concat

In [ ]:
from_teen.to_csv("teen.csv")
from_modern_poems.to_csv("modern_poems.csv")
from_dica_poems.to_csv("dica_poems.csv")

In [ ]:
train_data = pd.concat([from_teen, from_modern_poems])
train_data.to_csv("train.csv")

# 5. 기타 전처리
인터넷 게시판의 시를 크롤링 했기에, 코드 외적으로 수정할 부분들이 많았습니다.<br>
____, ----, ++++, 등으로 시작하는 분리(?) 마커 뒤에 개인적인 시에 대한 코멘트를 덧붙이는 경우,<br>
dica_poems는 작가명(개인정보)이 같이 크롤링 된 경우,<br>
주석을 단어에 *, 1) 등의 표시를 해둔 후 글 마지막에 설명하는 경우 등이 있었습니다.<br>
또한, 현대시 형식으로 띄어쓰기 없이 작성된 시들, 특수문자만 사용한 시들 등은 상당부분 직접 확인하고 삭제했습니다.